In [2]:
pip install SpeechRecognition pyaudio

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install pyaudio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import speech_recognition as sr
import json
import os
import time
import csv
from datetime import datetime
import re

class SpeechDataEntry:
    def __init__(self, forms_dir="form_templates", data_dir="entered_data"):
        """Initialize the speech data entry system with directories for forms and saved data"""
        self.recognizer = sr.Recognizer()
        self.forms_dir = forms_dir
        self.data_dir = data_dir
        
        # Create directories if they don't exist
        for directory in [forms_dir, data_dir]:
            if not os.path.exists(directory):
                os.makedirs(directory)
        
        # Create a sample form if none exists
        if not os.listdir(forms_dir):
            self._create_sample_form()
            
        # Command keywords
        self.keywords = {
            "exit": ["exit", "quit", "stop"],
            "repeat": ["repeat", "say again", "what was that"],
            "skip": ["skip", "next field", "next"],
            "help": ["help", "commands", "what can i say"],
            "undo": ["undo", "go back", "previous"],
            "save": ["save", "submit", "finish", "complete"]
        }
    
    def _create_sample_form(self):
        """Create a sample form template"""
        sample_form = {
            "name": "Customer Information Form",
            "fields": [
                {
                    "id": "full_name",
                    "type": "text",
                    "label": "Full Name",
                    "required": True,
                    "validation": None
                },
                {
                    "id": "email",
                    "type": "email",
                    "label": "Email Address",
                    "required": True,
                    "validation": r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
                },
                {
                    "id": "phone",
                    "type": "phone",
                    "label": "Phone Number",
                    "required": False,
                    "validation": r"^\d{10}$|^\d{3}[-\s]?\d{3}[-\s]?\d{4}$"
                },
                {
                    "id": "birth_date",
                    "type": "date",
                    "label": "Date of Birth (MM/DD/YYYY)",
                    "required": False,
                    "validation": r"^(0[1-9]|1[0-2])/(0[1-9]|[12][0-9]|3[01])/\d{4}$"
                },
                {
                    "id": "address",
                    "type": "text",
                    "label": "Home Address",
                    "required": False,
                    "validation": None
                }
            ]
        }
        
        with open(os.path.join(self.forms_dir, "customer_info.json"), 'w') as f:
            json.dump(sample_form, f, indent=4)
    
    def list_available_forms(self):
        """List all available form templates"""
        forms = []
        for file in os.listdir(self.forms_dir):
            if file.endswith(".json"):
                forms.append(file.replace(".json", ""))
                
        return forms
    
    def recognize_speech(self, prompt=None):
        """Recognize speech from microphone"""
        with sr.Microphone() as source:
            self.recognizer.adjust_for_ambient_noise(source, duration=0.5)
            
            if prompt:
                print(prompt)
                
            print("Listening...")
            try:
                audio = self.recognizer.listen(source, timeout=10, phrase_time_limit=10)
                print("Processing speech...")
                
                text = self.recognizer.recognize_google(audio)
                print(f"You said: {text}")
                return text.lower()
                
            except sr.WaitTimeoutError:
                print("No speech detected. Please try again.")
                return None
            except sr.UnknownValueError:
                print("Could not understand audio. Please try again.")
                return None
            except sr.RequestError as e:
                print(f"Could not request results; {e}")
                return None
            except Exception as e:
                print(f"Error: {e}")
                return None
    
    def validate_field(self, field, value):
        """Validate field input against validation pattern"""
        if not value:
            return field["required"] is False
            
        if field["validation"] and not re.match(field["validation"], value):
            return False
            
        return True
    
    def process_command(self, command):
        """Process voice commands"""
        if not command:
            return None
            
        command = command.lower()
        
        # Check for exit command
        for keyword in self.keywords["exit"]:
            if keyword in command:
                return "exit"
                
        # Check for repeat command
        for keyword in self.keywords["repeat"]:
            if keyword in command:
                return "repeat"
                
        # Check for skip command
        for keyword in self.keywords["skip"]:
            if keyword in command:
                return "skip"
                
        # Check for help command
        for keyword in self.keywords["help"]:
            if keyword in command:
                return "help"
                
        # Check for undo command
        for keyword in self.keywords["undo"]:
            if keyword in command:
                return "undo"
                
        # Check for save command
        for keyword in self.keywords["save"]:
            if keyword in command:
                return "save"
                
        # If no command matched, return the speech as data
        return command
    
    def show_help(self):
        """Display available voice commands"""
        print("\n=== Available Voice Commands ===")
        print("• 'Repeat' - Repeat the current field prompt")
        print("• 'Skip' or 'Next field' - Skip the current field")
        print("• 'Undo' or 'Go back' - Return to previous field")
        print("• 'Save' or 'Submit' - Complete and save the form")
        print("• 'Exit' or 'Quit' - Exit without saving")
        print("• 'Help' - Show this help menu")
        print("================================\n")
    
    def fill_form(self, form_name):
        """Fill out a form using voice commands"""
        form_path = os.path.join(self.forms_dir, f"{form_name}.json")
        
        if not os.path.exists(form_path):
            print(f"Form '{form_name}' not found.")
            return
        
        # Load form template
        with open(form_path, 'r') as f:
            form = json.load(f)
            
        print(f"\n=== Starting form: {form['name']} ===")
        print("Say 'Help' at any time to see available commands.")
        time.sleep(1)
            
        form_data = {}
        current_field_idx = 0
        
        while current_field_idx < len(form["fields"]):
            field = form["fields"][current_field_idx]
            
            # Prompt for the current field
            prompt = f"\nField {current_field_idx + 1}/{len(form['fields'])}: {field['label']}"
            if field["required"]:
                prompt += " (Required)"
            print(prompt)
            
            # Get voice input
            speech_text = self.recognize_speech("Please speak your answer or command:")
            command = self.process_command(speech_text)
            
            # Process command or input
            if command == "exit":
                if self._confirm_exit():
                    print("Exiting form without saving.")
                    return None
                else:
                    continue
                    
            elif command == "repeat":
                continue
                
            elif command == "help":
                self.show_help()
                continue
                
            elif command == "skip":
                if field["required"]:
                    print("This field is required and cannot be skipped.")
                    continue
                else:
                    form_data[field["id"]] = ""
                    current_field_idx += 1
                    continue
                    
            elif command == "undo":
                if current_field_idx > 0:
                    current_field_idx -= 1
                    # Remove the field data if we're going back
                    if form["fields"][current_field_idx]["id"] in form_data:
                        del form_data[form["fields"][current_field_idx]["id"]]
                    print(f"Going back to: {form['fields'][current_field_idx]['label']}")
                else:
                    print("Already at the first field.")
                continue
                
            elif command == "save":
                # Check if all required fields are filled
                missing_fields = []
                for f in form["fields"]:
                    if f["required"] and (f["id"] not in form_data or not form_data[f["id"]]):
                        missing_fields.append(f["label"])
                
                if missing_fields:
                    print("Cannot save: The following required fields are missing:")
                    for field_name in missing_fields:
                        print(f"- {field_name}")
                    continue
                else:
                    self._save_form_data(form, form_data)
                    return form_data
            
            else:
                # Process as field data
                if self.validate_field(field, command):
                    form_data[field["id"]] = command
                    current_field_idx += 1
                else:
                    # Special handling for specific field types
                    if field["type"] == "email":
                        print("Please provide a valid email address.")
                    elif field["type"] == "phone":
                        print("Please provide a valid phone number.")
                    elif field["type"] == "date":
                        print("Please provide a date in MM/DD/YYYY format.")
                    else:
                        print("Invalid input. Please try again.")
    
    def _confirm_exit(self):
        """Confirm if user wants to exit without saving"""
        print("Are you sure you want to exit without saving? Say 'yes' to confirm or anything else to continue.")
        confirmation = self.recognize_speech()
        return confirmation and "yes" in confirmation.lower()
    
    def _save_form_data(self, form, form_data):
        """Save form data to CSV file"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        form_name = form["name"].replace(" ", "_").lower()
        filename = f"{form_name}_{timestamp}.csv"
        file_path = os.path.join(self.data_dir, filename)
        
        # Create CSV with headers and data
        with open(file_path, 'w', newline='') as f:
            writer = csv.writer(f)
            
            # Write headers
            headers = ["Field", "Value"]
            writer.writerow(headers)
            
            # Write data
            for field in form["fields"]:
                field_id = field["id"]
                value = form_data.get(field_id, "")
                writer.writerow([field["label"], value])
        
        print(f"\nForm data saved successfully to: {filename}")
        return file_path
    
    def create_custom_form(self):
        """Create a custom form template through voice input"""
        print("\n=== Create Custom Form Template ===")
        
        # Get form name
        form_name = None
        while not form_name:
            print("What would you like to name this form?")
            form_name = self.recognize_speech()
            if not form_name:
                print("Could not understand the form name. Please try again.")
        
        # Initialize form structure
        form = {
            "name": form_name.title(),
            "fields": []
        }
        
        print(f"\nCreating form: {form['name']}")
        print("Now let's add fields to your form.")
        
        adding_fields = True
        field_num = 1
        
        while adding_fields:
            print(f"\n--- Field #{field_num} ---")
            
            # Get field label
            field_label = None
            while not field_label:
                field_label = self.recognize_speech("What should this field be labeled?")
                if not field_label:
                    print("Could not understand the field label. Please try again.")
            
            # Get field type
            field_type = None
            valid_types = ["text", "email", "phone", "date", "number"]
            
            print(f"What type of field is '{field_label}'? Options: {', '.join(valid_types)}")
            while field_type not in valid_types:
                field_type_input = self.recognize_speech()
                if field_type_input:
                    for valid_type in valid_types:
                        if valid_type in field_type_input.lower():
                            field_type = valid_type
                            break
                    
                    if not field_type:
                        print(f"Please specify a valid field type: {', '.join(valid_types)}")
                else:
                    print("Could not understand. Please try again.")
            
            # Is field required?
            print(f"Is the field '{field_label}' required? Say 'yes' or 'no'.")
            required_input = self.recognize_speech()
            required = required_input and "yes" in required_input.lower()
            
            # Create field object
            field_id = field_label.lower().replace(" ", "_")
            field = {
                "id": field_id,
                "type": field_type,
                "label": field_label,
                "required": required,
                "validation": self._get_validation_pattern(field_type)
            }
            
            # Add field to form
            form["fields"].append(field)
            field_num += 1
            
            # Add another field?
            print("\nWould you like to add another field? Say 'yes' or 'no'.")
            another_field = self.recognize_speech()
            adding_fields = another_field and "yes" in another_field.lower()
        
        # Save the form template
        form_filename = form["name"].lower().replace(" ", "_")
        with open(os.path.join(self.forms_dir, f"{form_filename}.json"), 'w') as f:
            json.dump(form, f, indent=4)
        
        print(f"\nForm template '{form['name']}' created successfully with {len(form['fields'])} fields.")
        return form_filename
    
    def _get_validation_pattern(self, field_type):
        """Return validation regex pattern based on field type"""
        if field_type == "email":
            return r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
        elif field_type == "phone":
            return r"^\d{10}$|^\d{3}[-\s]?\d{3}[-\s]?\d{4}$"
        elif field_type == "date":
            return r"^(0[1-9]|1[0-2])/(0[1-9]|[12][0-9]|3[01])/\d{4}$"
        elif field_type == "number":
            return r"^-?\d+(\.\d+)?$"
        else:
            return None


def main():
    """Main function to run the speech data entry system"""
    try:
        system = SpeechDataEntry()
        
        print("=== Speech-Based Data Entry System ===")
        print("This system allows you to fill out forms using voice commands.")
        
        while True:
            print("\nWhat would you like to do?")
            print("1. Fill out a form")
            print("2. Create a custom form template")
            print("3. List available forms")
            print("4. Exit")
            
            choice = input("Enter your choice (1-4): ")
            
            if choice == "1":
                forms = system.list_available_forms()
                if not forms:
                    print("No forms available. Please create a form first.")
                    continue
                    
                print("Available forms:")
                for i, form in enumerate(forms, 1):
                    print(f"{i}. {form}")
                
                form_idx = int(input(f"Select a form (1-{len(forms)}): ")) - 1
                if 0 <= form_idx < len(forms):
                    system.fill_form(forms[form_idx])
                else:
                    print("Invalid selection.")
                    
            elif choice == "2":
                system.create_custom_form()
                
            elif choice == "3":
                forms = system.list_available_forms()
                if forms:
                    print("\nAvailable forms:")
                    for i, form in enumerate(forms, 1):
                        print(f"{i}. {form}")
                else:
                    print("No forms available. Please create a form first.")
                    
            elif choice == "4":
                print("Exiting the speech data entry system. Goodbye!")
                break
                
            else:
                print("Invalid choice. Please try again.")
                
    except KeyboardInterrupt:
        print("\nProgram interrupted. Exiting...")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


if __name__ == "__main__":
    main()                      

=== Speech-Based Data Entry System ===
This system allows you to fill out forms using voice commands.

What would you like to do?
1. Fill out a form
2. Create a custom form template
3. List available forms
4. Exit


Enter your choice (1-4):  1


Available forms:
1. customer_info


Select a form (1-1):  1



=== Starting form: Customer Information Form ===
Say 'Help' at any time to see available commands.

Field 1/5: Full Name (Required)
Please speak your answer or command:
Listening...
Processing speech...
You said: hello

Field 2/5: Email Address (Required)
Please speak your answer or command:
Listening...
Processing speech...
You said: hello
Please provide a valid email address.

Field 2/5: Email Address (Required)
Please speak your answer or command:
Listening...
Processing speech...
You said: hello
Please provide a valid email address.

Field 2/5: Email Address (Required)
Please speak your answer or command:
Listening...
Processing speech...
You said: hello
Please provide a valid email address.

Field 2/5: Email Address (Required)
Please speak your answer or command:
Listening...
Processing speech...
You said: hello
Please provide a valid email address.

Field 2/5: Email Address (Required)
Please speak your answer or command:
Listening...
Processing speech...


In [ ]:
import speech_recognition as sr
import json
import os
import time
import csv
from datetime import datetime
import re

class SpeechDataEntry:
    def __init__(self, forms_dir="form_templates", data_dir="entered_data"):
        """Initialize the speech data entry system with directories for forms and saved data"""
        self.recognizer = sr.Recognizer()
        self.forms_dir = forms_dir
        self.data_dir = data_dir
        
        # Create directories if they don't exist
        for directory in [forms_dir, data_dir]:
            if not os.path.exists(directory):
                os.makedirs(directory)
        
        # Create a sample form if none exists
        if not os.listdir(forms_dir):
            self._create_sample_form()
            
        # Command keywords
        self.keywords = {
            "exit": ["exit", "quit", "stop"],
            "repeat": ["repeat", "say again", "what was that"],
            "skip": ["skip", "next field", "next"],
            "help": ["help", "commands", "what can i say"],
            "undo": ["undo", "go back", "previous"],
            "save": ["save", "submit", "finish", "complete"]
        }
    
    def _create_sample_form(self):
        """Create a sample form template"""
        sample_form = {
            "name": "Customer Information Form",
            "fields": [
                {
                    "id": "full_name",
                    "type": "text",
                    "label": "Full Name",
                    "required": True,
                    "validation": None
                },
                {
                    "id": "email",
                    "type": "email",
                    "label": "Email Address",
                    "required": True,
                    "validation": r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
                },
                {
                    "id": "phone",
                    "type": "phone",
                    "label": "Phone Number",
                    "required": False,
                    "validation": r"^\d{10}$|^\d{3}[-\s]?\d{3}[-\s]?\d{4}$"
                },
                {
                    "id": "birth_date",
                    "type": "date",
                    "label": "Date of Birth (MM/DD/YYYY)",
                    "required": False,
                    "validation": r"^(0[1-9]|1[0-2])/(0[1-9]|[12][0-9]|3[01])/\d{4}$"
                },
                {
                    "id": "address",
                    "type": "text",
                    "label": "Home Address",
                    "required": False,
                    "validation": None
                }
            ]
        }
        
        with open(os.path.join(self.forms_dir, "customer_info.json"), 'w') as f:
            json.dump(sample_form, f, indent=4)
    
    def list_available_forms(self):
        """List all available form templates"""
        forms = []
        for file in os.listdir(self.forms_dir):
            if file.endswith(".json"):
                forms.append(file.replace(".json", ""))
                
        return forms
    
    def recognize_speech(self, prompt=None):
        """Recognize speech from microphone"""
        with sr.Microphone() as source:
            self.recognizer.adjust_for_ambient_noise(source, duration=0.5)
            
            if prompt:
                print(prompt)
                
            print("Listening...")
            try:
                audio = self.recognizer.listen(source, timeout=10, phrase_time_limit=10)
                print("Processing speech...")
                
                text = self.recognizer.recognize_google(audio)
                print(f"You said: {text}")
                return text.lower()
                
            except sr.WaitTimeoutError:
                print("No speech detected. Please try again.")
                return None
            except sr.UnknownValueError:
                print("Could not understand audio. Please try again.")
                return None
            except sr.RequestError as e:
                print(f"Could not request results; {e}")
                return None
            except Exception as e:
                print(f"Error: {e}")
                return None
    
    def validate_field(self, field, value):
        """Validate field input against validation pattern"""
        if not value:
            return field["required"] is False
            
        if field["validation"] and not re.match(field["validation"], value):
            return False
            
        return True
    
    def process_command(self, command):
        """Process voice commands"""
        if not command:
            return None
            
        command = command.lower()
        
        # Check for exit command
        for keyword in self.keywords["exit"]:
            if keyword in command:
                return "exit"
                
        # Check for repeat command
        for keyword in self.keywords["repeat"]:
            if keyword in command:
                return "repeat"
                
        # Check for skip command
        for keyword in self.keywords["skip"]:
            if keyword in command:
                return "skip"
                
        # Check for help command
        for keyword in self.keywords["help"]:
            if keyword in command:
                return "help"
                
        # Check for undo command
        for keyword in self.keywords["undo"]:
            if keyword in command:
                return "undo"
                
        # Check for save command
        for keyword in self.keywords["save"]:
            if keyword in command:
                return "save"
                
        # If no command matched, return the speech as data
        return command
    
    def show_help(self):
        """Display available voice commands"""
        print("\n=== Available Voice Commands ===")
        print("• 'Repeat' - Repeat the current field prompt")
        print("• 'Skip' or 'Next field' - Skip the current field")
        print("• 'Undo' or 'Go back' - Return to previous field")
        print("• 'Save' or 'Submit' - Complete and save the form")
        print("• 'Exit' or 'Quit' - Exit without saving")
        print("• 'Help' - Show this help menu")
        print("================================\n")
    
    def fill_form(self, form_name):
        """Fill out a form using voice commands"""
        form_path = os.path.join(self.forms_dir, f"{form_name}.json")
        
        if not os.path.exists(form_path):
            print(f"Form '{form_name}' not found.")
            return
        
        # Load form template
        with open(form_path, 'r') as f:
            form = json.load(f)
            
        print(f"\n=== Starting form: {form['name']} ===")
        print("Say 'Help' at any time to see available commands.")
        time.sleep(1)
            
        form_data = {}
        current_field_idx = 0
        
        while current_field_idx < len(form["fields"]):
            field = form["fields"][current_field_idx]
            
            # Prompt for the current field
            prompt = f"\nField {current_field_idx + 1}/{len(form['fields'])}: {field['label']}"
            if field["required"]:
                prompt += " (Required)"
            print(prompt)
            
            # Get voice input
            speech_text = self.recognize_speech("Please speak your answer or command:")
            command = self.process_command(speech_text)
            
            # Process command or input
            if command == "exit":
                if self._confirm_exit():
                    print("Exiting form without saving.")
                    return None
                else:
                    continue
                    
            elif command == "repeat":
                continue
                
            elif command == "help":
                self.show_help()
                continue
                
            elif command == "skip":
                if field["required"]:
                    print("This field is required and cannot be skipped.")
                    continue
                else:
                    form_data[field["id"]] = ""
                    current_field_idx += 1
                    continue
                    
            elif command == "undo":
                if current_field_idx > 0:
                    current_field_idx -= 1
                    # Remove the field data if we're going back
                    if form["fields"][current_field_idx]["id"] in form_data:
                        del form_data[form["fields"][current_field_idx]["id"]]
                    print(f"Going back to: {form['fields'][current_field_idx]['label']}")
                else:
                    print("Already at the first field.")
                continue
                
            elif command == "save":
                # Check if all required fields are filled
                missing_fields = []
                for f in form["fields"]:
                    if f["required"] and (f["id"] not in form_data or not form_data[f["id"]]):
                        missing_fields.append(f["label"])
                
                if missing_fields:
                    print("Cannot save: The following required fields are missing:")
                    for field_name in missing_fields:
                        print(f"- {field_name}")
                    continue
                else:
                    self._save_form_data(form, form_data)
                    return form_data
            
            else:
                # Process as field data
                if self.validate_field(field, command):
                    form_data[field["id"]] = command
                    current_field_idx += 1
                else:
                    # Special handling for specific field types
                    if field["type"] == "email":
                        print("Please provide a valid email address.")
                    elif field["type"] == "phone":
                        print("Please provide a valid phone number.")
                    elif field["type"] == "date":
                        print("Please provide a date in MM/DD/YYYY format.")
                    else:
                        print("Invalid input. Please try again.")
    
    def _confirm_exit(self):
        """Confirm if user wants to exit without saving"""
        print("Are you sure you want to exit without saving? Say 'yes' to confirm or anything else to continue.")
        confirmation = self.recognize_speech()
        return confirmation and "yes" in confirmation.lower()
    
    def _save_form_data(self, form, form_data):
        """Save form data to CSV file"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        form_name = form["name"].replace(" ", "_").lower()
        filename = f"{form_name}_{timestamp}.csv"
        file_path = os.path.join(self.data_dir, filename)
        
        # Create CSV with headers and data
        with open(file_path, 'w', newline='') as f:
            writer = csv.writer(f)
            
            # Write headers
            headers = ["Field", "Value"]
            writer.writerow(headers)
            
            # Write data
            for field in form["fields"]:
                field_id = field["id"]
                value = form_data.get(field_id, "")
                writer.writerow([field["label"], value])
        
        print(f"\nForm data saved successfully to: {filename}")
        return file_path
    
    def create_custom_form(self):
        """Create a custom form template through voice input"""
        print("\n=== Create Custom Form Template ===")
        
        # Get form name
        form_name = None
        while not form_name:
            print("What would you like to name this form?")
            form_name = self.recognize_speech()
            if not form_name:
                print("Could not understand the form name. Please try again.")
        
        # Initialize form structure
        form = {
            "name": form_name.title(),
            "fields": []
        }
        
        print(f"\nCreating form: {form['name']}")
        print("Now let's add fields to your form.")
        
        adding_fields = True
        field_num = 1
        
        while adding_fields:
            print(f"\n--- Field #{field_num} ---")
            
            # Get field label
            field_label = None
            while not field_label:
                field_label = self.recognize_speech("What should this field be labeled?")
                if not field_label:
                    print("Could not understand the field label. Please try again.")
            
            # Get field type
            field_type = None
            valid_types = ["text", "email", "phone", "date", "number"]
            
            print(f"What type of field is '{field_label}'? Options: {', '.join(valid_types)}")
            while field_type not in valid_types:
                field_type_input = self.recognize_speech()
                if field_type_input:
                    for valid_type in valid_types:
                        if valid_type in field_type_input.lower():
                            field_type = valid_type
                            break
                    
                    if not field_type:
                        print(f"Please specify a valid field type: {', '.join(valid_types)}")
                else:
                    print("Could not understand. Please try again.")
            
            # Is field required?
            print(f"Is the field '{field_label}' required? Say 'yes' or 'no'.")
            required_input = self.recognize_speech()
            required = required_input and "yes" in required_input.lower()
            
            # Create field object
            field_id = field_label.lower().replace(" ", "_")
            field = {
                "id": field_id,
                "type": field_type,
                "label": field_label,
                "required": required,
                "validation": self._get_validation_pattern(field_type)
            }
            
            # Add field to form
            form["fields"].append(field)
            field_num += 1
            
            # Add another field?
            print("\nWould you like to add another field? Say 'yes' or 'no'.")
            another_field = self.recognize_speech()
            adding_fields = another_field and "yes" in another_field.lower()
        
        # Save the form template
        form_filename = form["name"].lower().replace(" ", "_")
        with open(os.path.join(self.forms_dir, f"{form_filename}.json"), 'w') as f:
            json.dump(form, f, indent=4)
        
        print(f"\nForm template '{form['name']}' created successfully with {len(form['fields'])} fields.")
        return form_filename
    
    def _get_validation_pattern(self, field_type):
        """Return validation regex pattern based on field type"""
        if field_type == "email":
            return r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
        elif field_type == "phone":
            return r"^\d{10}$|^\d{3}[-\s]?\d{3}[-\s]?\d{4}$"
        elif field_type == "date":
            return r"^(0[1-9]|1[0-2])/(0[1-9]|[12][0-9]|3[01])/\d{4}$"
        elif field_type == "number":
            return r"^-?\d+(\.\d+)?$"
        else:
            return None


def main():
    """Main function to run the speech data entry system"""
    try:
        system = SpeechDataEntry()
        
        print("=== Speech-Based Data Entry System ===")
        print("This system allows you to fill out forms using voice commands.")
        
        while True:
            print("\nWhat would you like to do?")
            print("1. Fill out a form")
            print("2. Create a custom form template")
            print("3. List available forms")
            print("4. Exit")
            
            choice = input("Enter your choice (1-4): ")
            
            if choice == "1":
                forms = system.list_available_forms()
                if not forms:
                    print("No forms available. Please create a form first.")
                    continue
                    
                print("Available forms:")
                for i, form in enumerate(forms, 1):
                    print(f"{i}. {form}")
                
                form_idx = int(input(f"Select a form (1-{len(forms)}): ")) - 1
                if 0 <= form_idx < len(forms):
                    system.fill_form(forms[form_idx])
                else:
                    print("Invalid selection.")
                    
            elif choice == "2":
                system.create_custom_form()
                
            elif choice == "3":
                forms = system.list_available_forms()
                if forms:
                    print("\nAvailable forms:")
                    for i, form in enumerate(forms, 1):
                        print(f"{i}. {form}")
                else:
                    print("No forms available. Please create a form first.")
                    
            elif choice == "4":
                print("Exiting the speech data entry system. Goodbye!")
                break
                
            else:
                print("Invalid choice. Please try again.")
                
    except KeyboardInterrupt:
        print("\nProgram interrupted. Exiting...")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


if __name__ == "__main__":
    main()                      

=== Speech-Based Data Entry System ===
This system allows you to fill out forms using voice commands.

What would you like to do?
1. Fill out a form
2. Create a custom form template
3. List available forms
4. Exit


Enter your choice (1-4):  2



=== Create Custom Form Template ===
What would you like to name this form?
Listening...
Processing speech...
You said: 1234

Creating form: 1234
Now let's add fields to your form.

--- Field #1 ---
What should this field be labeled?
Listening...
Processing speech...
You said: 5678
What type of field is '5678'? Options: text, email, phone, date, number
Listening...
Processing speech...
You said: ABCD
Please specify a valid field type: text, email, phone, date, number
Listening...
Processing speech...
You said: text
Is the field '5678' required? Say 'yes' or 'no'.
Listening...
Processing speech...
Could not understand audio. Please try again.

Would you like to add another field? Say 'yes' or 'no'.
Listening...
Processing speech...
Could not understand audio. Please try again.

Form template '1234' created successfully with 1 fields.

What would you like to do?
1. Fill out a form
2. Create a custom form template
3. List available forms
4. Exit


Enter your choice (1-4):  3



Available forms:
1. 1234
2. customer_info

What would you like to do?
1. Fill out a form
2. Create a custom form template
3. List available forms
4. Exit
